In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/task-uj/TASK.xlsx


In [4]:
!pip install bert-extractive-summarizer

  Created wheel for bert-extractive-summarizer: filename=bert_extractive_summarizer-0.4.2-py3-none-any.whl size=13711 sha256=cf966972aab955e7a1af62567b140ca868acfb49c30242b76089a19307ac996d
  Stored in directory: /root/.cache/pip/wheels/72/dc/37/8f4d3e5a337d48b296ebe21196d99dd5a52d494411b5ff9825
Successfully built bert-extractive-summarizer


In [3]:
!pip install transformers==2.2.0


     |████████████████████████████████| 360 kB 4.6 MB/s eta 0:00:01
ERROR: allennlp 1.0.0 has requirement transformers<2.12,>=2.9, but you'll have transformers 2.2.0 which is incompatible.
  Attempting uninstall: transformers
    Found existing installation: transformers 2.11.0
    Uninstalling transformers-2.11.0:
      Successfully uninstalled transformers-2.11.0


In [5]:
!pip install spacy==2.0.12

     |████████████████████████████████| 22.0 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 6.2 MB/s  eta 0:00:01
     |████████████████████████████████| 1.2 MB 40.9 MB/s eta 0:00:01
     |████████████████████████████████| 150 kB 43.5 MB/s eta 0:00:01
     |████████████████████████████████| 601 kB 49.6 MB/s eta 0:00:01
     |████████████████████████████████| 443 kB 41.9 MB/s eta 0:00:01
  Created wheel for spacy: filename=spacy-2.0.12-cp37-cp37m-linux_x86_64.whl size=28968885 sha256=4359667ce0765be2ff0ff7bd00726f3f932d2e8e4c671fc4afeae3904244d4ad
  Stored in directory: /root/.cache/pip/wheels/ed/69/44/1692fc0439f275181a07d05fb939a1b2ad482640e3b4824078
  Created wheel for murmurhash: filename=murmurhash-0.28.0-cp37-cp37m-linux_x86_64.whl size=42895 sha256=3babb92b4f88ed9bfba3ea7f027eef637edee9b64a965b3a57abdf34f00df086
  Stored in directory: /root/.cache/pip/wheels/b7/c8/d5/7b7ffd0ba48aa3f5630c0153b08edf10c8400f0933ff5b477d
  Created wheel for cymem: filename=cyme

    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [6]:
paragraphs = pd.read_excel('/kaggle/input/task-uj/TASK.xlsx', headers=True) #read dataset 
paragraphs['introduction'] = paragraphs['Unnamed: 1'] #cleanup column
paragraphs = paragraphs.drop(columns=['TEST DATASET','Unnamed: 1'],axis=1)
paragraphs = paragraphs.drop(paragraphs.index[0])#cleanup rows
new_index = list(range(0,len(paragraphs)))#reorder index
paragraphs['index'] = new_index
paragraphs = paragraphs.set_index('index')
print(len(paragraphs))
paragraphs.head()

1000


,introduction
index,
0,Acnesol Gel is an antibiotic that fights bacte...
1,Ambrodil Syrup is used for treating various re...
2,Augmentin 625 Duo Tablet is a penicillin-type ...
3,Azithral 500 Tablet is an antibiotic used to t...
4,Alkasol Oral Solution is a medicine used in th...


In [7]:
from summarizer import Summarizer,TransformerSummarizer

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


XLNet

In [8]:
#get input for the desired word count of the summary
sum_len=int(input('Enter length of summary? For better performance min_lenght is 60 '))

#initialize variables
summary = [] # for storing summaries

model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")

#generating summaries using ransformerSummarizer 
for i in range(0,len(paragraphs)):
        summary.append(model(paragraphs.introduction[i],min_length=sum_len))

#adding the summary to dataframe
paragraphs['summary'] = summary


#print top 5 results
#paragraphs.to_excel("output.xlsx")
paragraphs.head()

Enter length of summary? For better performance min_lenght is 60 60


,introduction,summary
index,,
0,Acnesol Gel is an antibiotic that fights bacte...,"It is used to treat acne, which appears as spo..."
1,Ambrodil Syrup is used for treating various re...,Ambrodil Syrup is used for treating various re...
2,Augmentin 625 Duo Tablet is a penicillin-type ...,Augmentin 625 Duo Tablet is a penicillin-type ...
3,Azithral 500 Tablet is an antibiotic used to t...,Azithral 500 Tablet is an antibiotic used to t...
4,Alkasol Oral Solution is a medicine used in th...,Alkasol Oral Solution is a medicine used in th...


GPT2

In [9]:
#get input for the desired word count of the summary
sum_len=int(input('Enter length of summary? For better performance min_lenght is 60 '))

#initialize variables
summary = [] # for storing summaries

GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")

#generating summaries using gensim summarizer #bert_summary = ''.join(bert_model(introduction, min_length=60))
for i in range(0,len(paragraphs)):
        summary.append(GPT2_model(paragraphs.introduction[i],min_length=sum_len))

#adding the summary to the main dataframe
paragraphs['summary'] = summary

#print top 5 results
#execute the following code to output the code as an excel file marked as 'output.xlsx'
#paragraphs.to_excel("output.xlsx")
paragraphs.head()

Enter length of summary? For better performance min_lenght is 60 60


,introduction,summary
index,,
0,Acnesol Gel is an antibiotic that fights bacte...,"It is used to treat acne, which appears as spo..."
1,Ambrodil Syrup is used for treating various re...,Ambrodil Syrup is used for treating various re...
2,Augmentin 625 Duo Tablet is a penicillin-type ...,Augmentin 625 Duo Tablet is a penicillin-type ...
3,Azithral 500 Tablet is an antibiotic used to t...,Azithral 500 Tablet is an antibiotic used to t...
4,Alkasol Oral Solution is a medicine used in th...,Alkasol Oral Solution is a medicine used in th...


BERT

In [10]:
#get input for the desired word count of the summary
sum_len=int(input('What is the desired word count of the summary? Ideal length is around 50 '))

#initialize variables
summary = [] # for storing summaries

bert_model = Summarizer()

#generating summaries using gensim summarizer 
for i in range(0,len(paragraphs)):

        
        summary.append(bert_model(paragraphs.introduction[i],min_length=sum_len))

#adding the summary to the main dataframe
paragraphs['summary'] = summary

#execute the following code to output the code as an excel file marked as 'output.xlsx'
#paragraphs.to_excel("output.xlsx")
paragraphs.head()

What is the desired word count of the summary? Ideal length is around 50 60


,introduction,summary
index,,
0,Acnesol Gel is an antibiotic that fights bacte...,"It is used to treat acne, which appears as spo..."
1,Ambrodil Syrup is used for treating various re...,Ambrodil Syrup is used for treating various re...
2,Augmentin 625 Duo Tablet is a penicillin-type ...,Augmentin 625 Duo Tablet is a penicillin-type ...
3,Azithral 500 Tablet is an antibiotic used to t...,Azithral 500 Tablet is an antibiotic used to t...
4,Alkasol Oral Solution is a medicine used in th...,Alkasol Oral Solution is a medicine used in th...


output

In [27]:
paragraphs.to_excel("/kaggle/working/output11.xlsx")